In [2]:
import os
import numpy as np
import pandas as pd
import tifffile as tif
import matplotlib.pyplot as plt

from transformers import RobertaTokenizer, RobertaModel
import torch

In [54]:
root_compounds_metadata = "/raid/cian/user/sergei.pnev/data/JUMP/cpg0004-lincs/metadata/compounds_metadata"

In [11]:
root = "/raid/cian/user/sergei.pnev/data/JUMP/cpg0004-lincs/metadata"
df = pd.read_csv(root + "/df4CancerCompounds.csv")
df_dmso = pd.read_csv(root + "/df_withDMSO_withCrops_filtered_v3.csv")
df_dmso_sample = df_dmso[df_dmso["Metadata_broad_sample"] == "DMSO"].sample(1).reset_index(drop=True)
df = pd.concat((df, df_dmso_sample)).reset_index(drop=True)

/tmp/ipykernel_829053/3676037683.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dmso = pd.read_csv(root + "/df_withDMSO_withCrops_filtered_v3.csv")


In [15]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Metadata_Plate', 'Metadata_Well',
       'Metadata_Site', 'Metadata_broad_sample', 'Metadata_moa',
       'Metadata_mmoles_per_liter', 'Metadata_dose_recode', 'RNA', 'ER', 'AGP',
       'Mito', 'DNA', 'Concentration', 'Treatment_ID', 'Compound',
       'pert_iname', 'Treatment_Replicate', 'Treatment', 'Plate_Map_Name',
       'Split', 'Metadata_broad_sample_ID', 'Metadata_moa_ID',
       'Metadata_Plate_ID', 'replicate_ID', 'combined_paths', 'Unique_well',
       'crops'],
      dtype='object')

In [13]:
tokenizer = RobertaTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model = RobertaModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
model.eval()

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/9.43k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/3.21k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/501 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/179M [00:00<?, ?B/s]

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(767, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropou

model.safetensors:   0%|          | 0.00/179M [00:00<?, ?B/s]

In [16]:
def smiles_to_embedding(smiles, tokenizer, model):
    # Tokenize the SMILES
    inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use the CLS token embedding as the representation
    cls_embedding = outputs.last_hidden_state[:, 0, :]  # shape: (batch_size, hidden_dim)
    return cls_embedding.squeeze().numpy() 

In [29]:
df_smiles = pd.read_csv("/raid/cian/user/sergei.pnev/data/JUMP/cpg0004-lincs/metadata/compounds_metadata/JUMP-Target-1_compound_metadata.tsv", sep='\t')

In [41]:
df_smiles.loc[df_smiles.index[-1], "broad_sample"] = "DMSO"

In [72]:
df_smiles

,broad_sample,InChIKey,pert_iname,pubchem_cid,gene,pert_type,control_type,smiles
0,BRD-A86665761-001-01-1,TZDUHAJSIBHXDL-UHFFFAOYSA-N,gabapentin-enacarbil,9883933.0,CACNB4,trt,NaN,CC(C)C(=O)OC(C)OC(=O)NCC1(CC(O)=O)CCCCC1
1,BRD-A22032524-074-09-9,HTIQEAQVCYTUBX-UHFFFAOYSA-N,amlodipine,2162.0,CACNA2D3,trt,NaN,CCOC(=O)C1=C(COCCN)NC(C)=C(C1c1ccccc1Cl)C(=O)OC
2,BRD-A01078468-001-14-8,PBBGSZCBWVPOOL-UHFFFAOYSA-N,hexestrol,3606.0,AKR1C1,trt,NaN,CCC(C(CC)c1ccc(O)cc1)c1ccc(O)cc1
3,BRD-K48278478-001-01-2,LOUPRKONTZGTKE-AFHBHXEDSA-N,quinine,94175.0,KCNN4,trt,NaN,COc1ccc2nccc([C@@H](O)[C@H]3C[C@@H]4CC[N@]3C[C...
4,BRD-K36574127-001-01-3,NYNZQNWKBKUAII-KBXCAEBGSA-N,LOXO-101,46188928.0,NTRK1,trt,NaN,O[C@H]1CCN(C1)C(=O)Nc1cnn2ccc(nc12)N1CCC[C@@H]...
...,...,...,...,...,...,...,...,...
302,BRD-K24616672-003-20-1,MFDFERRIHVXMIY-UHFFFAOYSA-N,procaine,4914.0,HTR3A,trt,NaN,CCN(CC)CCOC(=O)c1ccc(N)cc1
303,BRD-A82396632-008-30-8,BYBLEWFAAKGYCD-UHFFFAOYSA-N,miconazole,4189.0,KCNN1,trt,NaN,Clc1ccc(COC(Cn2ccnc2)c2ccc(Cl)cc2Cl)c(Cl)c1
304,BRD-K61250553-003-30-6,RDOIQAHITMMDAJ-UHFFFAOYSA-N,loperamide,3955.0,OPRM1,trt,NaN,CN(C)C(=O)C(CCN1CCC(O)(CC1)c1ccc(Cl)cc1)(c1ccc...
305,BRD-K70358946-001-17-3,CEUORZQYGODEFX-UHFFFAOYSA-N,aripiprazole,60795.0,HTR3A,trt,NaN,Clc1cccc(N2CCN(CCCCOc3ccc4CCC(=O)Nc4c3)CC2)c1Cl


In [49]:
smiles_dict = {k:v for k, v in zip(df_smiles["broad_sample"], df_smiles["smiles"])}

In [50]:
smiles_dict

{'BRD-A86665761-001-01-1': 'CC(C)C(=O)OC(C)OC(=O)NCC1(CC(O)=O)CCCCC1',
 'BRD-A22032524-074-09-9': 'CCOC(=O)C1=C(COCCN)NC(C)=C(C1c1ccccc1Cl)C(=O)OC',
 'BRD-A01078468-001-14-8': 'CCC(C(CC)c1ccc(O)cc1)c1ccc(O)cc1',
 'BRD-K48278478-001-01-2': 'COc1ccc2nccc([C@@H](O)[C@H]3C[C@@H]4CC[N@]3C[C@@H]4C=C)c2c1',
 'BRD-K36574127-001-01-3': 'O[C@H]1CCN(C1)C(=O)Nc1cnn2ccc(nc12)N1CCC[C@@H]1c1cc(F)ccc1F',
 'BRD-K74913225-001-14-0': 'CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(cc12)S(N)(=O)=O',
 'BRD-K94342292-001-01-3': 'FC(F)(F)c1cc(NC(=S)Nc2ccc(Br)cc2-c2nnn[nH]2)cc(c1)C(F)(F)F',
 'BRD-K19975102-001-01-2': 'Cc1ccc(NC(=O)Nc2cc(ccc2F)C(F)(F)F)cc1Nc1ccc2c(NC(=O)\\C2=C/c2ccc[nH]2)c1',
 'BRD-K25244359-066-02-6': 'O=C(Nc1ccc(cc1)C1(CCCC1)C#N)c1cccnc1NCc1ccncc1',
 'BRD-K18131774-001-18-4': 'CCOc1ccc2nc(sc2c1)S(N)(=O)=O',
 'BRD-A85242401-001-11-5': 'OC[C@H](O)[C@H]1OC(=O)C(=O)C1O',
 'BRD-K06426971-001-02-7': 'Cc1nc2c(s1)C(=O)C=C(Nc1ccc(C)cc1)C2=O',
 'BRD-K47049295-300-03-1': 'O=C(NCCCN1CCC2(CC1)OCc1ccccc21)[C@H]1CCCN1Cc1

In [62]:
df_smiles_v2 = pd.read_csv(root_compounds_metadata + "/repurposing_samples_20200324.txt", sep="\t", comment="!",
    encoding="ISO-8859-1")

In [63]:
df_smiles_v2

,broad_id,pert_iname,qc_incompatible,purity,vendor,catalog_no,vendor_name,expected_mass,smiles,InChIKey,pubchem_cid,deprecated_broad_id
0,BRD-K76022557-003-28-9,(R)-(-)-apomorphine,0,98.90,MedChemEx,HY-12723A,Apomorphine (hydrochloride hemihydrate),267.126,CN1CCc2cccc-3c2[C@H]1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-CYBMUJFWSA-N,6005.0,NaN
1,BRD-K76022557-003-02-7,(R)-(-)-apomorphine,0,97.34,Tocris,2073,(R)-(-)-Apomorphine hydrochloride,267.126,CN1CCc2cccc-3c2[C@H]1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-CYBMUJFWSA-N,6005.0,NaN
2,BRD-K76022557-003-29-9,(R)-(-)-apomorphine,0,97.36,Tocris,2073,(R)-(-)-Apomorphine hydrochloride,267.126,CN1CCc2cccc-3c2[C@H]1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-CYBMUJFWSA-N,6005.0,NaN
3,BRD-K76022557-001-03-9,(R)-(-)-apomorphine,0,95.80,Selleck,S4350,R-(-)-Apomorphine HCl Hemihydrate,267.126,CN1CCc2cccc-3c2[C@H]1Cc1ccc(O)c(O)c-31,VMWNQDUVQKEIOC-CYBMUJFWSA-N,6005.0,NaN
4,BRD-K75516118-001-04-1,(R)-(-)-rolipram,0,93.92,Tocris,1349,(R)-(-)-Rolipram,275.152,COc1ccc(cc1OC1CCCC1)[C@@H]1CNC(=O)C1,HJORMJIFDVBMOB-LBPRGKRZSA-N,448055.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13548,BRD-K63068307-001-10-9,ZSTK-474,0,96.15,MedChemEx,HY-50847,ZSTK474,417.172,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,HGVNLRPZOWWDKD-UHFFFAOYSA-N,11647372.0,NaN
13549,BRD-K63068307-001-08-9,ZSTK-474,0,98.66,Selleck,S1072,ZSTK474,417.172,FC(F)c1nc2ccccc2n1-c1nc(nc(n1)N1CCOCC1)N1CCOCC1,HGVNLRPZOWWDKD-UHFFFAOYSA-N,11647372.0,NaN
13550,BRD-K28761384-001-10-9,zuclopenthixol,0,94.95,MedChemEx,HY-A0163,Zuclopenthixol,400.138,OCCN1CCN(CC\C=C2\c3ccccc3Sc3ccc(Cl)cc23)CC1,WFPIAZLQTJBIFN-DVZOWYKESA-N,5311507.0,NaN
13551,BRD-K28761384-300-01-0,zuclopenthixol,0,97.19,Prestwick,Prestw-998,Zuclopenthixol dihydrochloride,400.138,OCCN1CCN(CC\C=C2\c3ccccc3Sc3ccc(Cl)cc23)CC1,WFPIAZLQTJBIFN-DVZOWYKESA-N,5311507.0,NaN


In [85]:
for k, v in zip(df_smiles_v2["broad_id"], df_smiles_v2["smiles"]):
    smiles_dict[k] = v

In [93]:
compounds

array(['BRD-K75295174-001-05-0', 'BRD-K99113996-001-02-0',
       'BRD-K80343549-001-02-6', 'BRD-K82244583-001-01-3', 'DMSO'],
      dtype=object)

In [86]:
compounds = df["Metadata_broad_sample"].unique()
smiles = [smiles_dict[i] for i in compounds]

In [87]:
smiles

['COc1cc(Nc2ncc3CN=C(c4cc(Cl)ccc4-c3n2)c2c(F)cccc2OC)ccc1C(O)=O |c:11|',
 'CNC(=O)c1cccc(c1)-c1ccc2c(nc(nc2n1)N1CCOC[C@@H]1C)N1CCOC[C@@H]1C',
 'CC(C)(O)CC(=O)NCCn1ccc2ncnc(Nc3ccc(Oc4cccc(c4)C(F)(F)F)c(Cl)c3)c12',
 'Cn1cnc2c(F)c(Nc3ccc(Br)cc3F)c(cc12)C(=O)NOCCO',
 'CS(=O)C']

In [88]:
embeddings = [smiles_to_embedding(i, tokenizer, model) for i in smiles]

In [89]:
len(embeddings)

5

In [91]:
feature_cols = [f"feature_{i}" for i in range(768)]
len(embeddings[0])

768

In [92]:
df_smiles = pd.DataFrame(embeddings, columns=feature_cols)

In [94]:
df_smiles["smiles"] = smiles
df_smiles["compounds"] = compounds

In [ ]:
df_smiles.to_csv(root + "/compounds_metadata/dfS.csv")

In [71]:
df_base = pd.read_csv(root + "/df_withDMSO_withCrops_filtered_v3.csv")

/tmp/ipykernel_829053/2076168050.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_base = pd.read_csv(root + "/df_withDMSO_withCrops_filtered_v3.csv")


In [74]:
compounds = list(df_base["Metadata_broad_sample"].unique())

In [77]:
smiles_dict

{'BRD-A86665761-001-01-1': 'CC(C)C(=O)O[C@@H](C)OC(=O)NCC1(CC(O)=O)CCCCC1 |&1:6,r|',
 'BRD-A22032524-074-09-9': 'CCOC(=O)C1=C(COCCN)NC(C)=C(C1c1ccccc1Cl)C(=O)OC',
 'BRD-A01078468-001-14-8': 'CCC(C(CC)c1ccc(O)cc1)c1ccc(O)cc1',
 'BRD-K48278478-001-01-2': 'COc1ccc2nccc([C@@H](O)[C@H]3C[C@@H]4CC[N@]3C[C@@H]4C=C)c2c1',
 'BRD-K36574127-001-01-3': 'O[C@H]1CCN(C1)C(=O)Nc1cnn2ccc(nc12)N1CCC[C@@H]1c1cc(F)ccc1F',
 'BRD-K74913225-001-14-0': 'CCN[C@H]1CN(CCCOC)S(=O)(=O)c2sc(cc12)S(N)(=O)=O',
 'BRD-K94342292-001-01-3': 'FC(F)(F)c1cc(NC(=S)Nc2ccc(Br)cc2-c2nn[nH]n2)cc(c1)C(F)(F)F',
 'BRD-K19975102-001-01-2': 'Cc1ccc(NC(=O)Nc2cc(ccc2F)C(F)(F)F)cc1Nc1ccc2\\C(=C\\c3ccc[nH]3)C(=O)Nc2c1',
 'BRD-K25244359-066-02-6': 'O=C(Nc1ccc(cc1)C1(CCCC1)C#N)c1cccnc1NCc1ccncc1',
 'BRD-K18131774-001-18-4': 'CCOc1ccc2nc(sc2c1)S(N)(=O)=O',
 'BRD-A85242401-001-11-5': 'OC[C@H](O)[C@H]1OC(=O)C(=O)[C@@H]1O |a:2,4,&1:10|',
 'BRD-K06426971-001-02-7': 'Cc1nc2c(s1)C(=O)C=C(Nc1ccc(C)cc1)C2=O |t:9|',
 'BRD-K47049295-300-03-1': 'O=C(N

In [82]:
smiles_dict["BRD-A74980173-001-11-9"] = "O(C)C1=C2N(C=C(C(O)=O)C(=O)C2=CC(F)=C1N3CC(C)NCC3)C4CC4"

smiles_dict["BRD-A69275535-001-01-5"] = "COC1C(C(C(C(C1O)O)O)O)O"

In [83]:
smiles = [smiles_dict[i] for i in compounds]
embeddings = [smiles_to_embedding(i, tokenizer, model) for i in smiles]

KeyError: 'BRD-K71480163-001-01-4'

In [84]:
for i in compounds:
    try:
        x = smiles_dict[i] 
    except:
        print(i)

BRD-K71480163-001-01-4
BRD-K81957469-001-01-0
BRD-K81258678-001-01-0
BRD-A68493689-001-01-9
BRD-A75172220-001-04-0
BRD-A75726477-003-21-3
BRD-A79768653-001-12-0
BRD-A78877355-001-03-0
BRD-A87479750-001-02-7
BRD-A88138582-001-04-8
BRD-K99257182-310-01-6
BRD-A97104540-004-15-0
BRD-A97674275-001-11-8
BRD-K97309399-001-09-4
BRD-A95939040-001-01-2
BRD-A92826379-001-01-5
BRD-A93424738-001-04-8
BRD-A93964801-003-03-3
BRD-K95260951-050-03-1
BRD-A95032015-065-01-2
BRD-A96060515-004-01-6
BRD-A84481105-003-20-6
BRD-A80151636-066-04-8
BRD-A86216746-046-02-6
BRD-K81332461-001-01-4
BRD-A82396632-008-29-0
BRD-K34008116-001-01-7
BRD-A50684349-003-02-9
BRD-K97688263-001-03-7
BRD-K89704198-001-12-3
BRD-A28746609-001-11-5
BRD-K31447894-236-01-0
BRD-A32172948-001-07-6
BRD-A33692381-001-01-9
BRD-A33084557-001-06-6
BRD-A33697453-008-02-4
BRD-K26011976-001-01-0
BRD-A26095496-001-04-0
BRD-A26690767-066-01-9
BRD-K28731095-001-01-7
BRD-A29731977-001-04-5
BRD-K30421593-050-04-9
BRD-A16665823-001-02-0
BRD-A186209

In [81]:
df_base[df_base["Metadata_broad_sample"] == "BRD-A69275535-001-01-5"].head()

,Unnamed: 0,Metadata_Plate,Metadata_Well,Metadata_Site,Metadata_broad_sample,Metadata_moa,Metadata_mmoles_per_liter,Metadata_dose_recode,RNA,ER,...,Treatment,Plate_Map_Name,Split,Metadata_broad_sample_ID,Metadata_moa_ID,Metadata_Plate_ID,replicate_ID,combined_paths,Unique_well,crops
315,315,SQ00014812,E13,1,BRD-A69275535-001-01-5,gamma secretase inhibitor,10.0,6,SQ00014812/r05c13f01p01-ch3sk1fk1fl1.tiff,SQ00014812/r05c13f01p01-ch2sk1fk1fl1.tiff,...,BRD-A69275535-001-01-5@10.0000000000000000,C-7161-01-LM6-022,Training,251,30,0,1.0,"SQ00014812/r05c13f01p01-ch1sk1fk1fl1.png,SQ000...",SQ00014812-E13,"[[14, 355], [12, 457], [14, 188], [21, 860], [..."
316,316,SQ00014812,E13,2,BRD-A69275535-001-01-5,gamma secretase inhibitor,10.0,6,SQ00014812/r05c13f02p01-ch3sk1fk1fl1.tiff,SQ00014812/r05c13f02p01-ch2sk1fk1fl1.tiff,...,BRD-A69275535-001-01-5@10.0000000000000000,C-7161-01-LM6-022,Training,251,30,0,1.0,"SQ00014812/r05c13f02p01-ch1sk1fk1fl1.png,SQ000...",SQ00014812-E13,"[[10, 92], [16, 231], [19, 18], [16, 431], [15..."
317,317,SQ00014812,E13,3,BRD-A69275535-001-01-5,gamma secretase inhibitor,10.0,6,SQ00014812/r05c13f03p01-ch3sk1fk1fl1.tiff,SQ00014812/r05c13f03p01-ch2sk1fk1fl1.tiff,...,BRD-A69275535-001-01-5@10.0000000000000000,C-7161-01-LM6-022,Training,251,30,0,1.0,"SQ00014812/r05c13f03p01-ch1sk1fk1fl1.png,SQ000...",SQ00014812-E13,"[[13, 682], [17, 394], [18, 816], [24, 678], [..."
318,318,SQ00014812,E13,4,BRD-A69275535-001-01-5,gamma secretase inhibitor,10.0,6,SQ00014812/r05c13f04p01-ch3sk1fk1fl1.tiff,SQ00014812/r05c13f04p01-ch2sk1fk1fl1.tiff,...,BRD-A69275535-001-01-5@10.0000000000000000,C-7161-01-LM6-022,Training,251,30,0,1.0,"SQ00014812/r05c13f04p01-ch1sk1fk1fl1.png,SQ000...",SQ00014812-E13,"[[21, 304], [21, 632], [25, 844], [22, 690], [..."
319,319,SQ00014812,E13,5,BRD-A69275535-001-01-5,gamma secretase inhibitor,10.0,6,SQ00014812/r05c13f05p01-ch3sk1fk1fl1.tiff,SQ00014812/r05c13f05p01-ch2sk1fk1fl1.tiff,...,BRD-A69275535-001-01-5@10.0000000000000000,C-7161-01-LM6-022,Training,251,30,0,1.0,"SQ00014812/r05c13f05p01-ch1sk1fk1fl1.png,SQ000...",SQ00014812-E13,"[[19, 39], [23, 340], [22, 574], [21, 90], [26..."
